# SQL ETL Notebook: Hellmouth Cup

Perform SQL ETL to load game and team data for the Hellmouth Cup and put it in the SQL database.

In [1]:
import os, sys, subprocess, json, time

In [2]:
from sqlalchemy import Column, Integer, String, Boolean, Table
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import DeclarativeBase

In [3]:
class Base(DeclarativeBase):
    pass

In [4]:
class HellmouthGame(Base):
    __tablename__ = "hellmouth"
    gameid = Column("gameid", String, primary_key=True)
    team1Abbr = Column("team1Abbr", String)
    team1Name = Column("team1Name", String)
    team1Color = Column("team1Color", String)
    team2Abbr = Column("team2Abbr", String)
    team2Name = Column("team2Name", String)
    team2Color = Column("team2Color", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    isPostseason = Column("isPostseason", Boolean)
    series = Column("series", String)
    league = Column("league", String)
    map_patternName = Column("map_patternName", String)
    map_mapName = Column("map_mapName", String)
    map_initialConditions1 = Column("map_initialConditions1", String)
    map_initialConditions2 = Column("map_initialConditions2", String)
    map_rows = Column("map_rows", Integer)
    map_columns = Column("map_columns", Integer)
    map_cellSize = Column("map_cellSize", Integer)
    team1Score = Column("team1Score", Integer)
    team2Score = Column("team2Score", Integer)
    generations = Column("generations", Integer)
    team1WinLoss_W = Column("team1WinLoss_W", Integer)
    team1WinLoss_L = Column("team1WinLoss_L", Integer)
    team2WinLoss_W = Column("team2WinLoss_W", Integer)
    team2WinLoss_L = Column("team2WinLoss_L", Integer)
    team1PostseasonWinLoss_W = Column("team1PostseasonWinLoss_W", Integer)
    team1PostseasonWinLoss_L = Column("team1PostseasonWinLoss_L", Integer)
    team2PostseasonWinLoss_W = Column("team2PostseasonWinLoss_W", Integer)
    team2PostseasonWinLoss_L = Column("team2PostseasonWinLoss_L", Integer)
    team1SeriesWinLoss_W = Column("team1SeriesWinLoss_W", Integer)
    team1SeriesWinLoss_L = Column("team1SeriesWinLoss_L", Integer)
    team2SeriesWinLoss_W = Column("team2SeriesWinLoss_W", Integer)
    team2SeriesWinLoss_L = Column("team2SeriesWinLoss_L", Integer)

This table class can also be used with the sqlalchemy CreateTable class, to generate the SQL needed to create this table.

Example from here: <https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html>

In [5]:
from sqlalchemy.schema import CreateTable
from sqlalchemy.dialects import sqlite

In [6]:
print(CreateTable(HellmouthGame.__table__).compile(dialect=sqlite.dialect()))


CREATE TABLE hellmouth (
	gameid VARCHAR NOT NULL, 
	"team1Abbr" VARCHAR, 
	"team1Name" VARCHAR, 
	"team1Color" VARCHAR, 
	"team2Abbr" VARCHAR, 
	"team2Name" VARCHAR, 
	"team2Color" VARCHAR, 
	season INTEGER, 
	day INTEGER, 
	"isPostseason" BOOLEAN, 
	series VARCHAR, 
	league VARCHAR, 
	"map_patternName" VARCHAR, 
	"map_mapName" VARCHAR, 
	"map_initialConditions1" VARCHAR, 
	"map_initialConditions2" VARCHAR, 
	map_rows INTEGER, 
	map_columns INTEGER, 
	"map_cellSize" INTEGER, 
	"team1Score" INTEGER, 
	"team2Score" INTEGER, 
	generations INTEGER, 
	"team1WinLoss_W" INTEGER, 
	"team1WinLoss_L" INTEGER, 
	"team2WinLoss_W" INTEGER, 
	"team2WinLoss_L" INTEGER, 
	"team1PostseasonWinLoss_W" INTEGER, 
	"team1PostseasonWinLoss_L" INTEGER, 
	"team2PostseasonWinLoss_W" INTEGER, 
	"team2PostseasonWinLoss_L" INTEGER, 
	"team1SeriesWinLoss_W" INTEGER, 
	"team1SeriesWinLoss_L" INTEGER, 
	"team2SeriesWinLoss_W" INTEGER, 
	"team2SeriesWinLoss_L" INTEGER, 
	PRIMARY KEY (gameid)
)




In [7]:
CUP = 'hellmouth'

def fetch_data(which_season0, fname):
    f = os.path.join('..', 'data', f'gollyx-{CUP}-data', f'season{which_season0}', fname)
    if not os.path.exists(f):
        raise Exception(f"Error: season {which_season0} not valid: {f} does not exist")
    with open(f, 'r') as fz:
        season0_seas = json.load(fz)
    return season0_seas

In [8]:
def fetch_season_data(which_season0):
    return fetch_data(which_season0, fname='season.json')

def fetch_postseason_data(which_season0):
    return fetch_data(which_season0, fname='postseason.json')

In [9]:
def flatten_season(season):
    games = []
    for day in season:
        for game in day:
            games.append(game)
    return games

def flatten_postseason(postseason):
    games = []
    for series in postseason:
        miniseason = postseason[series]
        for day in miniseason:
            for game in day:
                game['series'] = series
                games.append(game)
    return games

In [10]:
s3 = flatten_season(fetch_season_data(3))
g = s3[0]

In [11]:
print(g.keys())

dict_keys(['team1Name', 'team2Name', 'season', 'day', 'gameid', 'isPostseason', 'league', 'team1Color', 'team2Color', 'team1WinLoss', 'team2WinLoss', 'map', 'team1Score', 'team2Score', 'generations', 'team1Abbr', 'team2Abbr'])


In [12]:
p3 = flatten_postseason(fetch_postseason_data(3))
g2 = p3[0]

In [13]:
print(g2.keys())

dict_keys(['team1Name', 'team2Name', 'team1Color', 'team2Color', 'league', 'day', 'season', 'gameid', 'isPostseason', 'description', 'team1PostseasonWinLoss', 'team2PostseasonWinLoss', 'team1SeriesWinLoss', 'team2SeriesWinLoss', 'map', 'team1Score', 'team2Score', 'generations', 'team1Abbr', 'team2Abbr', 'series'])


In [14]:
def create_new_game(session, game, id_key='gameid'):
    this_id = game[id_key]
    
    # Check if game already exists
    existing_game = (
        session.query(HellmouthGame)
        .filter(HellmouthGame.gameid==this_id)
        .one_or_none()
    )
    if existing_game is not None:
        # print(f"Found an existing game with game id {this_id}, skipping insert")
        return

    # Now create the new game.
    # Keys are mostly one-to-one map, except maps and win/loss.
    # (First, account for the difference in keys in postseason vs regular season games)
    if 'description' not in game:
        game['description'] = ''
    if 'series' not in game:
        game['series'] = ''
    if 'league' not in game:
        game['league'] = ''
    if 'team1WinLoss' not in game:
        game['team1WinLoss'] = [-1, -1]
        game['team2WinLoss'] = [-1, -1]
    if 'team1PostseasonWinLoss' not in game:
        game['team1PostseasonWinLoss'] = [-1, -1]
        game['team2PostseasonWinLoss'] = [-1, -1]
    if 'team1SeriesWinLoss' not in game:
        game['team1SeriesWinLoss'] = [-1, -1]
        game['team2SeriesWinLoss'] = [-1, -1]
    
    hg = HellmouthGame(
        gameid = game['gameid'],
        team1Abbr = game['team1Abbr'],
        team1Name = game['team1Name'],
        team1Color = game['team1Color'],
        team2Abbr = game['team2Abbr'],
        team2Name = game['team2Name'],
        team2Color = game['team2Color'],
        season = game['season'],
        day = game['day'],
        isPostseason = game['isPostseason'],
        series = game['series'],
        league = game['league'],
        map_patternName = game['map']['patternName'],
        map_mapName = game['map']['mapName'],
        map_initialConditions1 = game['map']['initialConditions1'],
        map_initialConditions2 = game['map']['initialConditions2'],
        map_rows = game['map']['rows'],
        map_columns = game['map']['columns'],
        map_cellSize = game['map']['cellSize'],
        team1Score = game['team1Score'],
        team2Score = game['team2Score'],
        generations = game['generations'],
        team1WinLoss_W = game['team1WinLoss'][0],
        team1WinLoss_L = game['team1WinLoss'][1],
        team2WinLoss_W = game['team2WinLoss'][0],
        team2WinLoss_L = game['team2WinLoss'][1],
        team1PostseasonWinLoss_W = game['team1PostseasonWinLoss'][0],
        team1PostseasonWinLoss_L = game['team1PostseasonWinLoss'][1],
        team2PostseasonWinLoss_W = game['team2PostseasonWinLoss'][0],
        team2PostseasonWinLoss_L = game['team2PostseasonWinLoss'][1],
        team1SeriesWinLoss_W = game['team1SeriesWinLoss'][0],
        team1SeriesWinLoss_L = game['team1SeriesWinLoss'][1],
        team2SeriesWinLoss_W = game['team2SeriesWinLoss'][0],
        team2SeriesWinLoss_L = game['team2SeriesWinLoss'][1],
    )
    session.add(hg)

Now it's time to actually create a session, and call this function, and see if it works.

Documentation for creating and configuring an engine: <https://docs.sqlalchemy.org/en/20/core/engines.html#sqlite>

Documentation for creating a session: <https://docs.sqlalchemy.org/en/20/orm/session_basics.html>

In [15]:
import logging

logging.basicConfig()
logging.getLogger("sqlalchemy.engine").setLevel(logging.WARNING)
#logging.getLogger("sqlalchemy.pool").setLevel(logging.DEBUG)
logging.getLogger("sqlalchemy.pool").setLevel(logging.WARNING)

In [16]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

fpath = os.path.join(os.environ['HOME'], 'tmp', 'sqlite', 'mydb.db')
engine = create_engine(f"sqlite:///{fpath}")
Session = sessionmaker(engine)

In [17]:
# Start by creating all the tables
Base.metadata.create_all(engine)

In [18]:
# Run a simple query on the tables
with Session() as session:
    results = session.query(HellmouthGame).all()

print(len(results))

11042


In [19]:
# By using .begin(), each function call will call session.add() multiple times within the context,
# and when the context is closed, it will apply all changes.
with Session.begin() as session:
    create_new_game(session, s3[0])

## Load Season Data into SQLite Database

The code below loads all 24 regular season games into the SQLite database:

In [20]:
for season0 in range(24):
    sdat = flatten_season(fetch_season_data(season0))
    # Create a new context manager for each season, to limit number of inserts at one time
    with Session.begin() as session:
        print(f"Working on season0={season0}")
        for game in sdat:
            create_new_game(session, game)
print("\nDone!")

Working on season0=0
Working on season0=1
Working on season0=2
Working on season0=3
Working on season0=4
Working on season0=5
Working on season0=6
Working on season0=7
Working on season0=8
Working on season0=9
Working on season0=10
Working on season0=11
Working on season0=12
Working on season0=13
Working on season0=14
Working on season0=15
Working on season0=16
Working on season0=17
Working on season0=18
Working on season0=19
Working on season0=20
Working on season0=21
Working on season0=22
Working on season0=23

Done!


In [21]:
for season0 in range(24):
    pdat = flatten_postseason(fetch_postseason_data(season0))
    # Create a new context manager for each season, to limit number of inserts at one time
    with Session.begin() as session:
        print(f"Working on season0={season0} postseason")
        for game in pdat:
            create_new_game(session, game)
print("\nDone!")

Working on season0=0 postseason
Working on season0=1 postseason
Working on season0=2 postseason
Working on season0=3 postseason
Working on season0=4 postseason
Working on season0=5 postseason
Working on season0=6 postseason
Working on season0=7 postseason
Working on season0=8 postseason
Working on season0=9 postseason
Working on season0=10 postseason
Working on season0=11 postseason
Working on season0=12 postseason
Working on season0=13 postseason
Working on season0=14 postseason
Working on season0=15 postseason
Working on season0=16 postseason
Working on season0=17 postseason
Working on season0=18 postseason
Working on season0=19 postseason
Working on season0=20 postseason
Working on season0=21 postseason
Working on season0=22 postseason
Working on season0=23 postseason

Done!
